In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target
num_features = X.shape[1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Fit the model on the training data
gnb.fit(X_train, y_train)

out_onnx = "gaussian_nb.onnx"

initial_type = [('float_input', FloatTensorType([None, num_features]))]
onnx_model = convert_sklearn(gnb, initial_types=initial_type)

meta =  onnx_model.metadata_props.add()
meta.key = "sklearn_model"
meta.value = "GaussianNB"

with open(out_onnx, "wb") as f:
    f.write( onnx_model.SerializeToString())


In [19]:
import pandas as pd

# Predict on the test data
predictions = gnb.predict(X_test)

# Create a pandas DataFrame with feature columns
df = pd.DataFrame(X_test, columns=data.feature_names)
# Add columns for the true labels and the predictions
df['true_label'] = y_test
df['predicted_label'] = predictions

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),true_label,predicted_label
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
4,6.8,2.8,4.8,1.4,1,1
